In [4]:
!pip install opencv-python

  Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (63.0 MB)


In [ ]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

# Load pose data
pose_df = pd.read_csv("your_pose_data.csv")
pose_df['frame_time_ms'] = pd.to_numeric(pose_df['frame_time_ms'], errors='coerce')

# Sort for safety
pose_df = pose_df.sort_values(by="frame_time_ms").reset_index(drop=True)

# Load video
cap = cv2.VideoCapture("your_video.webm")
if not cap.isOpened():
    raise IOError("Cannot open video")

# Function to draw pose
def plot_pose_on_frame(frame, pose_row, ax=None):
    height, width = frame.shape[:2]
    landmark_count = len([c for c in pose_row.index if '_x' in c])

    xs = [pose_row[f'landmark_{i}_x'] * width for i in range(landmark_count)]
    ys = [pose_row[f'landmark_{i}_y'] * height for i in range(landmark_count)]

    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))
    ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    ax.set_axis_off()
    for x, y in zip(xs, ys):
        if not pd.isna(x) and not pd.isna(y):
            ax.add_patch(Circle((x, y), radius=4, color='red'))

# Find frame closest to a given pose timestamp
def get_closest_pose(time_ms, window=33):  # ~1 frame at 30fps
    diffs = (pose_df['frame_time_ms'] - time_ms).abs()
    if diffs.min() <= window:
        return pose_df.iloc[diffs.idxmin()]
    return None  # No close match


# Read frames and overlay closest pose
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get timestamp in ms for this frame
    frame_time_ms = cap.get(cv2.CAP_PROP_POS_MSEC)

    pose_row = get_closest_pose(frame_time_ms)

    fig, ax = plt.subplots(figsize=(8, 6))
    plot_pose_on_frame(frame, pose_row, ax=ax)
    ax.set_title(f"Frame Time: {frame_time_ms:.2f} ms\nPose Time: {pose_row['frame_time_ms']:.2f} ms")
    plt.show()

    # Optional: break after one frame for testing
    break

cap.release()


Frames: -274395318096429600, FPS: 29.75, Resolution: 320x240


ValueError: Number of samples, -274395318096429600, must be non-negative.

In [8]:
print(cv2.getBuildInformation())



General configuration for OpenCV 4.11.0 =====================================
  Version control:               4.11.0-dirty

  Platform:
    Timestamp:                   2025-01-16T09:53:30Z
    Host:                        Linux 5.15.0-1074-azure x86_64
    CMake:                       3.31.1
    CMake generator:             Unix Makefiles
    CMake build tool:            /bin/gmake
    Configuration:               Release
    Algorithm Hint:              ALGO_HINT_ACCURATE

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      AVX (8 files):             + SSSE3 SSE4_1 POPCNT SSE4_2 AVX
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 AVX FP16
      AVX2 (36 files):           + SSSE3 SSE4_1 POPCNT SSE4_2 AVX FP16 AVX2 FMA3
      AVX512_S